In [ ]:
!pip install gradio pillow opencv-python-headless numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [15]:

import gradio as gr
from PIL import Image, ImageEnhance
import numpy as np
import cv2
import re

def interpret_prompt(prompt):
    """Interpret user prompt and return corresponding adjustments."""
    adjustments = {
        "brightness": 1.0,
        "contrast": 1.0,
        "color": 1.0,
        "sharpness": 1.0,
        "blur": 0,
        "crop": None,
        "resize": None
    }


    if "brighten" in prompt:
        match = re.search(r'brighten by (\d+(\.\d+)?)', prompt)
        if match:
            adjustments["brightness"] = float(match.group(1))


    if "contrast" in prompt:
        match = re.search(r'contrast by (\d+(\.\d+)?)', prompt)
        if match:
            adjustments["contrast"] = float(match.group(1))


    if "color" in prompt:
        match = re.search(r'color by (\d+(\.\d+)?)', prompt)
        if match:
            adjustments["color"] = float(match.group(1))


    if "sharpen" in prompt:
        match = re.search(r'sharpen by (\d+(\.\d+)?)', prompt)
        if match:
            adjustments["sharpness"] = float(match.group(1))


    if "blur" in prompt:
        match = re.search(r'blur by (\d+)', prompt)
        if match:
            adjustments["blur"] = int(match.group(1))


    if "crop" in prompt:
        match = re.search(r'crop to (\d+), (\d+), (\d+), (\d+)', prompt)
        if match:
            adjustments["crop"] = (
                int(match.group(1)),
                int(match.group(2)),
                int(match.group(3)),
                int(match.group(4))
            )


    if "resize" in prompt:
        match = re.search(r'resize to (\d+), (\d+)', prompt)
        if match:
            adjustments["resize"] = (
                int(match.group(1)),
                int(match.group(2))
            )

    return adjustments

def adjust_brightness_contrast_color(image, brightness=1.0, contrast=1.0, color=1.0):
    """Adjust brightness, contrast, and color intensity of the image."""
    img = Image.fromarray(image)
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast)
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(color)
    return np.array(img)

def sharpen_image(image, sharpness=1.0):
    """Adjust sharpness of the image."""
    img = Image.fromarray(image)
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(sharpness)
    return np.array(img)

def blur_image(image, blur_intensity=0):
    """Apply blur to the image."""
    img = np.array(image)
    if blur_intensity > 0:
        img = cv2.GaussianBlur(img, (2 * blur_intensity + 1, 2 * blur_intensity + 1), 0)
    return img

def crop_image(image, top=0, left=0, height=100, width=100):
    """Crop the image to a specified region."""
    img = Image.fromarray(image)
    cropped_img = img.crop((left, top, left + width, top + height))
    return np.array(cropped_img)

def resize_image(image, new_width=100, new_height=100):
    """Resize the image to the specified width and height."""
    img = Image.fromarray(image)
    resized_img = img.resize((new_width, new_height))
    return np.array(resized_img)

def process_image(image, prompt):
    """Main function to apply edits based on prompt and return the result."""
    adjustments = interpret_prompt(prompt)


    edited_image = adjust_brightness_contrast_color(image,
                                                     adjustments["brightness"],
                                                     adjustments["contrast"],
                                                     adjustments["color"])


    edited_image = sharpen_image(edited_image, adjustments["sharpness"])


    edited_image = blur_image(edited_image, adjustments["blur"])


    if adjustments["crop"]:
        edited_image = crop_image(edited_image, *adjustments["crop"])


    if adjustments["resize"]:
        edited_image = resize_image(edited_image, *adjustments["resize"])

    return edited_image


image_input = gr.Image(type="numpy", label="Upload Image")
prompt_input = gr.Textbox(label="Enter your editing prompt", placeholder="e.g., 'brighten by 1.5'")

image_output = gr.Image(type="numpy", label="Edited Image")

iface = gr.Interface(
    fn=process_image,
    inputs=[image_input, prompt_input],
    outputs=image_output,
    title="AI Image Editing Application",
    description="Edit images using natural language prompts."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://925e0c4d4a90668dd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:

import gradio as gr
from PIL import Image, ImageEnhance
import numpy as np
import cv2


def adjust_brightness_contrast_color(image, brightness=1.0, contrast=1.0, color=1.0):
    """Adjust brightness, contrast, and color intensity of the image."""
    img = Image.fromarray(image)
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast)
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(color)
    return np.array(img)


def sharpen_image(image, sharpness=1.0):
    """Adjust sharpness of the image."""
    img = Image.fromarray(image)
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(sharpness)
    return np.array(img)


def blur_image(image, blur_intensity=0):
    """Apply blur to the image."""
    img = np.array(image)
    if blur_intensity > 0:
        img = cv2.GaussianBlur(img, (2 * blur_intensity + 1, 2 * blur_intensity + 1), 0)
    return img


def crop_image(image, top=0, left=0, height=100, width=100):
    """Crop the image to a specified region."""
    img = Image.fromarray(image)
    cropped_img = img.crop((left, top, left + width, top + height))
    return np.array(cropped_img)


def resize_image(image, new_width=100, new_height=100):
    """Resize the image to the specified width and height."""
    img = Image.fromarray(image)
    resized_img = img.resize((new_width, new_height))
    return np.array(resized_img)


def remove_background(image):
    """Remove background using basic thresholding as a demo (advanced models can be added)."""
    img = np.array(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    background_removed = cv2.bitwise_and(img, img, mask=mask)
    return background_removed



brightness_slider = gr.Slider(0.5, 2.0, value=1.0, label="Brightness")
contrast_slider = gr.Slider(0.5, 2.0, value=1.0, label="Contrast")
color_slider = gr.Slider(0.5, 2.0, value=1.0, label="Color Intensity")
sharpness_slider = gr.Slider(0.5, 2.0, value=1.0, label="Sharpness")
blur_slider = gr.Slider(0, 10, value=0, label="Blur Intensity")
top_crop_slider = gr.Slider(0, 500, value=0, label="Crop Top")
left_crop_slider = gr.Slider(0, 500, value=0, label="Crop Left")
height_crop_slider = gr.Slider(50, 500, value=100, label="Crop Height")
width_crop_slider = gr.Slider(50, 500, value=100, label="Crop Width")
resize_width_slider = gr.Slider(50, 1000, value=100, label="Resize Width")
resize_height_slider = gr.Slider(50, 1000, value=100, label="Resize Height")
image_input = gr.Image(type="numpy", label="Upload Image")
image_output = gr.Image(type="numpy", label="Edited Image")


def process_image(image, brightness, contrast, color, sharpness, blur_intensity, crop_top, crop_left, crop_height, crop_width, resize_width, resize_height):
    """Main function to apply edits and return the result."""

    edited_image = adjust_brightness_contrast_color(image, brightness, contrast, color)

    edited_image = sharpen_image(edited_image, sharpness)

    edited_image = blur_image(edited_image, blur_intensity)

    edited_image = crop_image(edited_image, crop_top, crop_left, crop_height, crop_width)

    edited_image = resize_image(edited_image, resize_width, resize_height)
    return edited_image


def bg_removal(image):
    """Remove background from the input image."""
    result = remove_background(image)
    return result



iface = gr.Interface(
    fn=process_image,
    inputs=[
        image_input,
        brightness_slider,
        contrast_slider,
        color_slider,
        sharpness_slider,
        blur_slider,
        top_crop_slider,
        left_crop_slider,
        height_crop_slider,
        width_crop_slider,
        resize_width_slider,
        resize_height_slider
    ],
    outputs=image_output,
    title="Advanced AI Image Editing Application",
    description="Adjust brightness, contrast, color, sharpness, blur. Crop and resize images.",
)

iface_bg = gr.Interface(
    fn=bg_removal,
    inputs=image_input,
    outputs=image_output,
    title="Background Removal Tool",
    description="Remove background from images using basic thresholding.",
)



iface.launch(share=True)
iface_bg.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ec3814f78f8029577.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4bc9a4c659b483b964.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
